In [26]:
# from classifier_lewidi import *
# from params_lewidi import *
from DataManager import *

import os
import argparse
from datetime import datetime
import pytz
import re


In [27]:


filepath1 = '../data/ConvAbuse_dataset/'
filepath2 = '../data/ArMIS_dataset/'
filepath3 = '../data/HS-Brexit_dataset/'
filepath4 = '../data/MD-Agreement_dataset/'




In [28]:

data = DataManager(filepath1, filepath2, filepath3, filepath4)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/nv2006/anaconda3/envs/lewidi/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_24805/2557284913.py", line 1, in <module>
    data = DataManager(filepath1, filepath2, filepath3, filepath4)
  File "/home/nv2006/scratch/lewidi_2023/code/DataManager.py", line 50, in __init__
    #         flag = 1
  File "/home/nv2006/scratch/lewidi_2023/code/DataManager.py", line 140, in annotation_annotator_split
    for ind in panda.index:
  File "/home/nv2006/anaconda3/envs/lewidi/lib/python3.10/site-packages/pandas/core/strings/accessor.py", line 129, in wrapper
    return func(self, *args, **kwargs)
  File "/home/nv2006/anaconda3/envs/lewidi/lib/python3.10/site-packages/pandas/core/strings/accessor.py", line 2515, in findall
    result = self._data.array._str_findall(pat, flags)
  File "/home/nv2006/anaconda3/envs/lewidi/lib/python3.10/site

In [ ]:
print(data)

In [ ]:
def_params = params_lewidi()

In [ ]:

key = "conv_train"

annotators = list(sorted(set(itertools.chain.from_iterable(data[key]['annotators'].str.findall("\w+")))))
    
model = ToxicityClassifier(data, annotators=annotators, params=def_params)



In [ ]:
score, results = model.CV()

score["params"] = ", ".join(key + ": " + str(val) for key, val in def_params.__dict__.items())
score["task"] = model.params.task

pacific = pytz.timezone('US/Pacific')
sa_time = datetime.now(pacific)
name_time = sa_time.strftime('%m%d%y-%H:%M')
score["time"] = name_time
print(score)

score_dir = os.path.join(def_params.source_dir, "results", "GHC", "classification.csv")
result_dir = os.path.join(def_params.source_dir, "results", "GHC", name_time + "_" + def_params.task + ".csv")
if os.path.exists(score_dir):
    pd.DataFrame.from_records([score]).to_csv(score_dir, header=False,  index=False, mode="a")
else:
    pd.DataFrame.from_records([score]).to_csv(score_dir, index=False)

    pd.DataFrame.from_dict(results).to_csv(result_dir, index=False)
